In [13]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import graphlab as gl
import random
import statsmodels.api as sm
import pylab as pl
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
gl.canvas.set_target('ipynb')
from datetime import datetime
%matplotlib inline
from collections import Counter
from pandas import ExcelWriter

number = preprocessing.LabelEncoder()
df_business=pd.read_csv('D:\\ds\\mini3\\data1\\yelp_dataset_challenge_academic_dataset\\yelp_academic_dataset_business.csv')
df_business=df_business[df_business['categories'].str.contains("u'Restaurants'")]

In [2]:
##Loading business Data
df_business=pd.read_csv('D:\\ds\\mini3\\data1\\yelp_dataset_challenge_academic_dataset\\yelp_academic_dataset_business.csv')
df_business=df_business[df_business['categories'].str.contains("u'Restaurants'")]
print df_business['business_id'].count()
print len(df_business['business_id'].unique())

21892
21892


In [14]:
###load review data
fields = ['business_id','date','stars']
data = pd.read_csv('D:\\ds\\mini3\\data1\\yelp_dataset_challenge_academic_dataset\\yelp_academic_dataset_review.csv', skipinitialspace=True, usecols=fields)

###Find the restraunt businesses in review data
dataResRev = data[data['business_id'].isin(df_business['business_id'])]
dataResRev
print dataResRev.count()
print len(dataResRev['business_id'].unique())

,business_id,stars,date
22,mVHrayjG3uZ_RLHkLj-AMg,5,2012-12-01
23,mVHrayjG3uZ_RLHkLj-AMg,5,2013-03-15
24,mVHrayjG3uZ_RLHkLj-AMg,5,2013-03-30
25,mVHrayjG3uZ_RLHkLj-AMg,4,2013-10-20
26,mVHrayjG3uZ_RLHkLj-AMg,5,2013-11-07
27,mVHrayjG3uZ_RLHkLj-AMg,5,2014-03-22
28,mVHrayjG3uZ_RLHkLj-AMg,4,2014-09-29
29,mVHrayjG3uZ_RLHkLj-AMg,5,2014-09-29
30,mVHrayjG3uZ_RLHkLj-AMg,5,2014-11-03
31,mVHrayjG3uZ_RLHkLj-AMg,4,2014-12-18


In [15]:
###Find last date of review for each business id
dataResRev['date'] = pd.to_datetime(dataResRev['date'])
data_review = dataResRev.iloc[dataResRev.groupby(['business_id']).apply(lambda x: x['date'].idxmax())]

data['date'] = pd.to_datetime(data['date'])
data_review = data.iloc[data.groupby(['business_id']).apply(lambda x: x['date'].idxmax())]
data_review.rename(columns={data_review.columns[2]:'end_date'}, inplace=True)
data_review.head(10)
print type(data['date'])
print data_review.count()
print len(data_review['business_id'].unique())

,business_id,stars,end_date
1399275,--1emggGHgoG6ipd_RMb-g,4,2014-06-13
870089,--4Pe8BZ6gj57VFL5mUE8g,5,2014-07-31
1241261,--5jkZ3-nUPZxUvtcbr8Uw,5,2014-12-27
11061,--7PRjnsjMA6uhPK8mW13Q,1,2013-11-17
1191779,--BlvDO_RG2yElKu9XA1_g,5,2014-12-04
1078669,--Dl2rW_xO8GuYBomlg9zw,5,2012-10-31
1258430,--NCcmQqh_NZbjW1S5XsGQ,5,2013-07-13
1021875,--Ol5mVSMaW8ExtmWRUmKA,5,2014-11-17
7929,--UE_y6auTgq3FXlvUMkbw,4,2015-01-08
991236,--WLj8iK9DUKxkPnMHAoeA,2,2014-11-18


In [16]:
###Set data as index and remove date column. This is now working set from review data
dataResRev = dataResRev.set_index(pd.DatetimeIndex(dataResRev['date']))
col_list= list(dataResRev)
col_list.remove('date')
dataResRev= dataResRev[col_list]
dataResRev

,business_id,stars
2012-12-01,mVHrayjG3uZ_RLHkLj-AMg,5
2013-03-15,mVHrayjG3uZ_RLHkLj-AMg,5
2013-03-30,mVHrayjG3uZ_RLHkLj-AMg,5
2013-10-20,mVHrayjG3uZ_RLHkLj-AMg,4
2013-11-07,mVHrayjG3uZ_RLHkLj-AMg,5
2014-03-22,mVHrayjG3uZ_RLHkLj-AMg,5
2014-09-29,mVHrayjG3uZ_RLHkLj-AMg,4
2014-09-29,mVHrayjG3uZ_RLHkLj-AMg,5
2014-11-03,mVHrayjG3uZ_RLHkLj-AMg,5
2014-12-18,mVHrayjG3uZ_RLHkLj-AMg,4


In [17]:
##Create the data table with 12 month aggregation
dataResRevT = dataResRev.groupby([pd.TimeGrouper('12M'), 'business_id']).agg(['mean', 'count'])
dataResRevT.reset_index(inplace = True)
dataResRevT.columns = [' '.join(col).strip() for col in dataResRevT.columns.values]

##drop extra column
col_list= list(dataResRevT)

##Not used now
##print col_list
##col_list.remove('index')

##cosmetic changes
feature_vector = dataResRevT[col_list]
feature_vector.rename(columns={feature_vector.columns[0]:'date',feature_vector.columns[2]:'stars',feature_vector.columns[3]:'reviews'}, inplace=True)

##get closed or open
feature_vector_1 = pd.merge(feature_vector,df_business[['business_id','open']],  on =['business_id'])

##get end review date along
feature_vector_2 = pd.merge(feature_vector_1,data_review[['business_id','end_date']],  on =['business_id'])

##Creating label for ML
feature_vector_2['end_date'] = pd.to_datetime(feature_vector_2['end_date'])
feature_vector_2['date'] = pd.to_datetime(feature_vector_2['date'])

feature_vector_2['label'] = -1
feature_vector_2['label'] = feature_vector_2['end_date'] - feature_vector_2['date']
feature_vector_2['label'] = feature_vector_2['label'].astype(pd.Timedelta).apply(lambda l: l.days)
feature_vector_2['label'] = feature_vector_2['label'] / 365
feature_vector_2['label'] = feature_vector_2['label'].astype(int)
feature_vector_2['label'][feature_vector_2['open'] == True] = -1
final_feature_vector = feature_vector_2[['label','stars','reviews']]
final_feature_vector.head()

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,label,stars,reviews
0,6,4.000000,1
1,5,4.500000,2
2,3,3.875000,8
3,2,4.111111,9
4,1,3.571429,7
5,0,3.333333,6
6,0,3.000000,6
7,7,5.000000,1
8,6,4.000000,2
9,4,4.000000,4


In [75]:
feature_vector_2['label'][feature_vector_2['open'] == True] = -1
final_feature_vector = feature_vector_2[['label','stars','reviews']]
final_feature_vector.head()

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,label,stars,reviews
0,6,4.000000,1
1,5,4.500000,2
2,3,3.875000,8
3,2,4.111111,9
4,1,3.571429,7


In [96]:
## trying to find only the closed
#feature_vector_2 = feature_vector_2[feature_vector_2['open'] == False]
#final_feature_vector = feature_vector_2[['label','stars','reviews']]
#final_feature_vector.head()
final_feature_vector.to_csv('D:\\ds\\final_projet\\time_series_feature_vector.csv')

In [90]:
## to select only 1/10th rows 
#rows = random.sample(final_feature_vector.index, 10)
#df_business = final_feature_vector.ix[rows]
## selecting all
df_business = final_feature_vector
data_train = np.copy(df_business.as_matrix(columns=df_business.columns[0:]))
print data_train


[[  6.           4.           1.        ]
 [  5.           4.5          2.        ]
 [  3.           3.875        8.        ]
 ..., 
 [  0.           4.           1.        ]
 [  0.           3.875       64.        ]
 [  0.           3.94545455  55.        ]]


In [91]:
###Split training and test sets

number_samples = len(data_train)
number_training_samples = 0
number_testing_samples = 0
correct_predicted = 0

opened = []
closed = []
training = []
testing = []
#for i in range(number_samples):
#    if(data_train[i][0] == -1):
#        opened.append(data_train[i]);
#    else:
#        closed.append(data_train[i]);

for i in range(number_samples):
    if(i % 4 != 0):
        opened.append(data_train[i]);
    else:
        closed.append(data_train[i]);


nOpened = len(opened)
nClosed = len(closed)
training = opened[0:(3*nOpened/4)] + closed[0:(3*nClosed/4)];
testing = opened[(3*nOpened/4)+1:nOpened] + closed[(3*nClosed/4)+1:nClosed];
random.shuffle(training)
random.shuffle(testing)

training = np.array(training)
testing = np.array(testing)

X_training = training[:, 1:]
Y_training = training[:,0].astype(np.int)
X_testing = testing[:, 1:]
Y_testing = testing[:,0].astype(np.int)


In [93]:
from sklearn import datasets
from sklearn.multiclass import OneVsOneClassifier
clf = RandomForestClassifier(n_jobs=2)
clf.fit(X_training, Y_training)

predicted = clf.predict(X_testing)
number_testing_samples = len(X_testing)
number_training_samples = len(X_training)

correct_predicted = accuracy_score(Y_testing, predicted, normalize=False)

print "Number of samples in training data", number_training_samples
print "Number of samples in testing data", number_testing_samples
print "Number of correct predicted testing samples are", correct_predicted
print "accuracy is" ,((float(correct_predicted) *100)/number_testing_samples)

Number of samples in training data 11304
Number of samples in testing data 3766
Number of correct predicted testing samples are 2719
accuracy is 72.1986192246


In [107]:
##Just to show the predicted label
for i in range(len(predicted)):
    if((predicted[i] != 0) & (predicted[i] != 1)):
        print predicted[i]      

3
2
2
2
3
2
2
5
3
2
2
5
2
4
3
5
4
3
2
2
3
2
3
5
6
5
4
3
2
2
2
2
4
3
2
3
2
2
2
6
5
4
3
2
3
2
4
2
2
6
5
4
2
2
3
4
2
3
3
5
2
5
4
3
2
4
2
3
3
2
2
2
2
5
4
3
2
2
3
2
2
2
3
2
3
2
2
2
2
5
4
3
2
2
2
4
3
2
3
2
3
4
4
3
2
2
2
2
2
4
3
5
4
2
4
3
2
2
4
3
2
3
2
2
3
2
2
5
4
3
2
2
2
2
3
5
2
2
2
3
2
4
3
2
2
2
3
2
2
2
2
3
3
2
6
4
3
2
2
2
5
4
3
2
2
3
2
2
3
2
2
3
2
2
3
3
2
2
3
2
4
2
4
2
2
2
2
4
2
3
3
2
2
2
3
4
3
2
2
3
2
2
3
2
2
2
2
2
2
3
3
2
2
3
2
5
3
2
2
2
2
2
2
2
2
2
4
3
2
5
2
2
3
5
4
3
2
4
3
2
3
2
2
3
4
2
2
3
3
2
3
2
3
3
5
2
2
4
2
2
3
2
2
4
2
2
5
4
3
2
3
2
2
2
2
4
3
2
2
2
2
2
3
2
3
4
2
2
3
4
3
2
3
2
3
3
2
3
3
2
5
4
3
2
2
2
3
3
2
2
3
2
2
2
3
2
3
2
2
2
3
2
3
3
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
5
4
3
2
2
2
2
4
3
2
4
3
2
2
2
2
4
3
2
3
2
2
2
3
2
2
3
2
4
3
2
2
2
2
3
2
2
3
2
2
3
3
2
2
3
3
2
2
3
2
3
3
3
3
2
2
3
2
4
3
3
2
4
2
2
2
3
2
3
3
2
2
4
3
2
2
2
2
2
2
2
2
5
2
4
2
3
2
2
2
2
3
3
2
2
3
2
2
2
2
2
3
2
2
3
2
3
2
3
2
2
2
3
2
2
2
2
2
2
3
2
4
2
2
2
2
2
2
4
2
4
2
2
2
2
3
2
2
2
2
2
2
3
4
3
2
2
2
4
2
